In [1]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("./plants_thermal_input_data_final.csv")

# Display the dataframe
df.head()

,NEONSite,FluxnetSite,Year,Month,NEON_PAR,GrowingSeason,MODIS_FPAR,MODIS_GPP,MODIS_NPP,MODIS_CUE,MODIS_ET,MODIS_PET,Ameriflux_Precipitation,Ameriflux_NEE,Ameriflux_RE,Ameriflux_GPP
0,ABBY,US-xAB,2019,1,234.266082,0,0.511250,15812.0,NaN,NaN,NaN,NaN,5.525,0.985518,1.87615,1.080930
1,ABBY,US-xAB,2019,2,158.791374,0,0.312857,NaN,NaN,NaN,NaN,NaN,8.037,1.062960,1.42750,0.354154
2,ABBY,US-xAB,2019,3,274.517711,1,0.588750,NaN,NaN,NaN,NaN,NaN,2.682,-0.093046,3.11090,3.265210
3,ABBY,US-xAB,2019,4,239.743986,1,0.370000,NaN,NaN,NaN,NaN,NaN,7.601,0.311318,4.95291,4.502570
4,ABBY,US-xAB,2019,5,371.709899,1,0.695000,NaN,NaN,NaN,NaN,NaN,2.044,-0.352640,6.97303,7.388660


In [2]:
import os
print("cwd:", os.getcwd())

cwd: /workspaces/Soil_respiration


In [3]:
!ls -la

total 32
drwxrwxrwx+ 4 codespace root       4096 Nov 17 18:19 .
drwxr-xrwx+ 5 codespace root       4096 Nov 17 18:06 ..
drwxrwxrwx+ 8 codespace root       4096 Nov 17 18:18 .git
drwxrwxrwx+ 2 codespace codespace  4096 Nov 17 18:19 .ipynb_checkpoints
-rw-rw-rw-  1 codespace root         63 Nov 17 18:07 README.md
-rw-rw-rw-  1 codespace codespace 11618 Nov 17 18:17 analysis.ipynb
